In [5]:
#Import Packages
from selenium import webdriver
import time
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup

In [6]:
#Import Packages
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [7]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

In [8]:
urls = ["https://www.quikr.com/jobs/india+it-software-developer+salary-20000rs+zwqxj3349828408"]
#urls = ["https://www.quikr.com/jobs/india+animation+salary-20000rs+3d-modeler_animator_graphic-designer_stop-motion-animator+zwqxj33498284081"]

In [31]:
d = {"Company_name":[], "Role":[],  "Skills":[], "Job_description":[],"Salary":[], "Location":[], "Application_link":[] }
print(d)

{'Company_name': [], 'Role': [], 'Skills': [], 'Job_description': [], 'Salary': [], 'Location': [], 'Application_link': []}


In [10]:
def get_role_name(soup):

    try:
        Job_name = soup.find('h1', class_='sc-dxhf6u-0 igItXx sc-11567zh-3 jtMGHc').text

    except AttributeError:
        Job_name = ""
    return Job_name

def get_company_name(soup):
    try:
        Company_name = soup.find('label', string='Hiring Company Name').find_next_sibling('span').text

    except AttributeError:
        Company_name = ""
    return Company_name

def get_location(soup):
    try:
        location = soup.find('label', class_='sc-1ew444r-5 bJxiZx fwB cD').text

    except AttributeError:
        location = ""
    return location

def get_jd(soup):
    try:
        description_div = soup.find('div', id='Description')

        # Extract description
        job_description = description_div.find_all('span')[0].get_text()

    except AttributeError:
        job_description = ""
    return job_description

def get_salary(soup):
    try:
        lower_salary = soup.find('label', class_='sc-dxhf6u-0 hoLrKb sc-11567zh-7 cJtIqi').text.split('₹')[1].split('\xa0')[0]

        # Extracting the upper salary
        upper_salary = soup.find('label', class_='sc-dxhf6u-0 hoLrKb sc-11567zh-7 cJtIqi').text.split('₹')[2]
        salary = lower_salary + '-' + upper_salary

    except AttributeError:
        salary = ""
    return salary

def get_skills(soup):
    try:
        description_div = soup.find('div', id='Description')
        skills = [span.get_text() for span in description_div.find_all('span')[5:]]

    except AttributeError:
        skills = ""
    return skills


In [15]:
def scroll(driver):
    previous_height = driver.execute_script('return document.body.scrollHeight') # Calculate the height of body
# Scroll down repeatedly
    for i in range(500): # Adjust the range if needed
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # Scroll 
        time.sleep(10)
        new_height = driver.execute_script('return document.body.scrollHeight') #Calculate height
        if new_height == previous_height:
            break
        previous_height = new_height
        time.sleep(1)

In [16]:
def get_job_details(driver):
    
    html_content = driver.page_source # Get the HTML content of the current page
    soup = BeautifulSoup(html_content, 'html.parser') # Parse the HTML content using BeautifulSoup
    links = soup.find_all('a', class_='job-title') # Find all links with the specified class
    links_list = []
    for link in links:
        links_list.append(link.get('href'))
        
    for link in links_list:
            new_webpage = requests.get(link, headers=HEADERS) # Send request to access the page
            new_soup = BeautifulSoup(new_webpage.content, "html.parser") # Parse the HTML content of the new webpage
            # Call functions to extract the details
            d["Company_name"].append(get_company_name(new_soup))
            d["Role"].append(get_role_name(new_soup))
            d["Location"].append(get_location(new_soup))
            d["Salary"].append(get_salary(new_soup))
            d["Skills"].append(get_skills(new_soup))
            d["Job_description"].append(get_jd(new_soup))
            d["Application_link"].append(link)
            print(get_role_name(new_soup))

In [17]:
for url in urls:
    # Initialize Chrome WebDriver
    driver = webdriver.Chrome()
    driver.implicitly_wait(2)  # Wait for up to 2 seconds for elements to appear
    try:
        # Open the URL
        driver.get(url)
        
        # Call function to scrape job details
        scroll(driver)
        get_job_details(driver)
    finally:
        # Close the WebDriver to free up resources
        driver.quit()

Looking for IT jobs ,  Hiring 2024 freshers for internship and placement
Requirement of fresher and experienced  software programmers and network engineer
IT job opportunity fresher eligible this position immediately join this process
Immediate requirement for IT Software
IMMEDIATELY HIRING FOR UI DEVELOPER HAVING 2+ YEARS OF EXPERIENCE IN CHENNAI
php developer
It software developer
Laraval  Developer
FULLSTACK DEVELOPER INTERN 2024
web developer
Sr. Software Developer C#.NET,C++MFC,SQL
.NET Developer
Sr Laravel developer
Full Stack Developer
Software Developer
JAVA Developer
Senior PHP Laravel Developer for SAAS-Based Product
Senior PHP Laravel Developer for SAAS-Based Product
PHP LARAVEL DEVELOPER
Web Developer
Angular developer
3D Artist
php developer
PHP Developer
Requirement For Laravel , Android, IOS
SOFTWARE TRAINER
ASP.NET DEVELOPER
Software Trainee
MEAN-stack Developer - MongoDB, Express, Angular & NodeJS
PHP Developer
WordPress Developer
Dot Net Developer
Flutter Developer
As

In [18]:
print(len(d["Job_description"]))

573


In [27]:
# Check the lengths of lists in the dictionary
lengths = {key: len(value) for key, value in d.items()}
print(lengths)

{'Company_name': 573, 'Role': 573, 'Skills': 573, 'Job_description': 573, 'Salary': 573, 'Location': 573, 'Application_link': 573}


In [28]:
df = pd.DataFrame.from_dict(d)

  # Save DataFrame to CSV file
csv_file_path = "quickr_software_developer.csv"
df.to_csv(csv_file_path, index=False)